In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn import preprocessing
from sklearn.preprocessing import minmax_scale

In [ ]:
path = 'data/'

# Meat Consumption

http://www.fao.org/faostat/en/?#data/FBS/visualize

In [ ]:
meat_consumption_path = path + 'FoodBalanceSheets_E_Europe_1.csv'
meat_consumption = pd.read_csv(meat_consumption_path, low_memory=False)

In [ ]:
years_in_dataset = range(1961, 2014)
years_to_keep = range(2004, 2014) # 2013 is the last year available in the dataset
years_to_drop = range(1961, 2004) # we don't use data before 2004

In [ ]:
columns_to_drop = ['Y' + str(x) + 'F' for x in years_in_dataset] # These columns doesn't contain relevant data
columns_to_drop += ['Y' + str(x) for x in years_to_drop]
meat_consumption = meat_consumption.drop(columns=columns_to_drop)

In [ ]:
rename_columns = ['Area Code', 'Area', 'Item Code', 'Item', 'Element Code', 'Element', 'Unit'] + [x for x in years_to_keep]

In [ ]:
rename_columns

In [ ]:
meat_consumption.columns = rename_columns

In [ ]:
# meat_consumption.Item.unique()
meat_related_item = ['Bovine Meat', 'Mutton & Goat Meat', 'Pigmeat', 'Poultry Meat', 'Meat, Other']

In [ ]:
meat_consumption = meat_consumption[meat_consumption.Item.isin(meat_related_item)]

In [ ]:
europe = [
    "Austria", 
    "Belgium", 
    "Bulgaria", 
    "Croatia", 
    "Republic of Cyprus", 
    "Czech Republic", 
    "Denmark", 
    "Estonia", 
    "Finland", 
    "France", 
    "Germany", 
    "Greece", 
    "Hungary", 
    "Ireland", 
    "Italy", 
    "Latvia", 
    "Lithuania", 
    "Luxembourg", 
    "Malta", 
    "Netherlands", 
    "Poland", 
    "Portugal", 
    "Romania", 
    "Slovakia", 
    "Slovenia", 
    "Spain", 
    "Sweden", 
    "United Kingdom"
]

In [ ]:
meat_consumption = meat_consumption[meat_consumption.Area.isin(europe)]

In [ ]:
meat_consumption_melted = pd.melt(meat_consumption, id_vars=["Area Code", "Area", "Item Code", "Item", 'Element Code' , 'Element', 'Unit'], 
             value_vars=years_to_keep)

In [ ]:
meat_consumption_melted.columns

In [ ]:
new_columns = ["Area Code", "Area", "Item Code", "Item", 'Element Code' , 'Element', 'Unit', 'years', 'value']

In [ ]:
meat_consumption_melted.columns = new_columns

In [ ]:
meat_consumption_melted = meat_consumption_melted[meat_consumption_melted['Element'] == 'Domestic supply quantity']

In [ ]:
meat_consumption_melted.Item.unique()

## All meats

In [ ]:
meat_consumption_melted.head()

In [ ]:
meat_consumption_melted = meat_consumption_melted.groupby(['years', 'Area'])['value'].sum().unstack(level=0)

meat_consumption_melted['Area'] = meat_consumption_melted.index

meat_consumption_melted = pd.melt(meat_consumption_melted, id_vars=['Area'], value_vars=years_to_keep)

In [ ]:
meat_consumption_melted.head()

In [ ]:
fig = px.line(meat_consumption_melted, x="years", y='value', color='Area')
fig.update_xaxes(title_text='Years')
fig.update_yaxes(title_text='All meats consumption per 1000 tons')
fig.show()

In [ ]:
meat_consumption_melted.value.describe()

In [ ]:
min_val = meat_consumption_melted.value.min()
max_val = meat_consumption_melted.value.max()

fig = px.choropleth(meat_consumption_melted, 
                    locations="Area", 
                    scope="europe", 
                    locationmode = 'country names', 
                    color='value', 
                    hover_name="Area", 
                    animation_frame="years", 
                    range_color=[min_val, 6500])
fig.show()

## Normalize all

In [ ]:
meat_consumption_melted_normalized = meat_consumption_melted.copy()

In [ ]:
# normalize value column for each country
meat_consumption_melted_normalized['value'] = meat_consumption_melted_normalized.groupby(['Area']).value.transform(lambda x: minmax_scale(x.astype(float)))

In [ ]:
fig = px.line(meat_consumption_melted_normalized, x="years", y='value', color='Area')
fig.update_xaxes(title_text='Years')
fig.update_yaxes(title_text='All meats consumption per 1000 tons (normalized per country)')
fig.show()

In [ ]:
min_val = meat_consumption_melted_normalized.value.min()
max_val = meat_consumption_melted_normalized.value.max()

fig = px.choropleth(meat_consumption_melted_normalized, 
                    locations="Area", 
                    scope="europe", 
                    locationmode = 'country names', 
                    color='value', 
                    hover_name="Area", 
                    animation_frame="years", 
                    range_color=[min_val, max_val])
fig.show()

## Poultry Meat 

In [ ]:
poultry_consumption = meat_consumption_melted[meat_consumption_melted['Item'] == 'Poultry Meat']

In [ ]:
# poultry_consumption = poultry_consumption[poultry_consumption['Element'] == 'Domestic supply quantity']

In [ ]:
fig = px.line(poultry_consumption, x="years", y='value', color='Area')
fig.update_xaxes(title_text='Years')
fig.update_yaxes(title_text='Poultry consumption in 1000 tons')
fig.show()

In [ ]:
poultry_consumption.value.describe()

In [ ]:
min_val = poultry_consumption.value.min()
max_val = poultry_consumption.value.max()

fig = px.choropleth(poultry_consumption, 
                    locations="Area", 
                    scope="europe", 
                    locationmode = 'country names', 
                    color='value', 
                    hover_name="Area", 
                    animation_frame="years", 
                    range_color=[min_val, max_val])
fig.show()